In [2]:
from keras import backend as K
import time
from multiprocessing.dummy import Pool
K.set_image_data_format('channels_first')
import cv2
import os
import glob
import numpy as np
from numpy import genfromtxt
import tensorflow as tf
from fr_utils import *
from inception_blocks_v2 import *
import time
PADDING = 50
ready_to_detect_identity = True

import subprocess

def say(text):
    subprocess.call(['say', text])

FRmodel = faceRecoModel(input_shape=(3, 96, 96))


Using TensorFlow backend.


In [3]:
def triplet_loss(y_true, y_pred, alpha = 0.3):
    """
    Implementation of the triplet loss as defined by formula (3)
    
    Arguments:
    y_pred -- python list containing three objects:
            anchor -- the encodings for the anchor images, of shape (None, 128)
            positive -- the encodings for the positive images, of shape (None, 128)
            negative -- the encodings for the negative images, of shape (None, 128)
    
    Returns:
    loss -- real number, value of the loss
    """
    
    anchor, positive, negative = y_pred[0], y_pred[1], y_pred[2]
    
    # Step 1: Compute the (encoding) distance between the anchor and the positive, you will need to sum over axis=-1
    pos_dist = tf.reduce_sum(tf.square(tf.subtract(anchor, positive)), axis=-1)
    # Step 2: Compute the (encoding) distance between the anchor and the negative, you will need to sum over axis=-1
    neg_dist = tf.reduce_sum(tf.square(tf.subtract(anchor, negative)), axis=-1)
    # Step 3: subtract the two previous distances and add alpha.
    basic_loss = tf.add(tf.subtract(pos_dist, neg_dist), alpha)
    # Step 4: Take the maximum of basic_loss and 0.0. Sum over the training examples.
    loss = tf.reduce_sum(tf.maximum(basic_loss, 0.0))
    
    return loss

FRmodel.compile(optimizer = 'adam', loss = triplet_loss, metrics = ['accuracy'])
load_weights_from_FaceNet(FRmodel)

In [4]:
def prepare_database():
    database = {}

    # load all the images of individuals to recognize into the database
    for file in glob.glob("images/*"):
        file = ''.join([i for i in file if not i.isdigit()])
        identity = os.path.splitext(os.path.basename(file))[0]
        if identity in database:
            database[identity] = database[identity].append(img_path_to_encoding(file, FRmodel))
        else:
            database[identity] = [img_path_to_encoding(file, FRmodel)]
    return database

def webcam_face_recognizer(database):
    """
    Runs a loop that extracts images from the computer's webcam and determines whether or not
    it contains the face of a person in our database.

    If it contains a face, an audio message will be played welcoming the user.
    If not, the program will process the next frame from the webcam
    """
    global ready_to_detect_identity

    cv2.namedWindow("preview")
    vc = cv2.VideoCapture(0)

    face_cascade = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')
    
    while vc.isOpened():
        _, frame = vc.read()
        img = frame

        # We do not want to detect a new identity while the program is in the process of identifying another person
        if ready_to_detect_identity:
            img = process_frame(img, frame, face_cascade)   
        
        key = cv2.waitKey(100)
        cv2.imshow("preview", img)

        if key == 27: # exit on ESC
            break
    cv2.destroyWindow("preview")

def process_frame(img, frame, face_cascade):
    """
    Determine whether the current frame contains the faces of people from our database
    """
    global ready_to_detect_identity
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray, 1.3, 5)

    # Loop through all the faces detected and determine whether or not they are in the database
    identities = []
    for (x, y, w, h) in faces:
        x1 = x-PADDING
        y1 = y-PADDING
        x2 = x+w+PADDING
        y2 = y+h+PADDING

        img = cv2.rectangle(frame,(x1, y1),(x2, y2),(255,0,0),2)

        identity = find_identity(frame, x1, y1, x2, y2)

        if identity is not None:
            identities.append(identity)

    if identities != []:
        cv2.imwrite('example.png',img)

        ready_to_detect_identity = False
        pool = Pool(processes=1) 
        # We run this as a separate process so that the camera feedback does not freeze
        pool.apply_async(welcome_users, [identities])
    return img

def find_identity(frame, x1, y1, x2, y2):
    """
    Determine whether the face contained within the bounding box exists in our database

    x1,y1_____________
    |                 |
    |                 |
    |_________________x2,y2

    """
    height, width, channels = frame.shape
    # The padding is necessary since the OpenCV face detector creates the bounding box around the face and not the head
    part_image = frame[max(0, y1):min(height, y2), max(0, x1):min(width, x2)]
    
    return who_is_it(part_image, database, FRmodel)

def who_is_it(image, database, model):
    """
    Implements face recognition for the happy house by finding who is the person on the image_path image.
    
    Arguments:
    image_path -- path to an image
    database -- database containing image encodings along with the name of the person on the image
    model -- your Inception model instance in Keras
    
    Returns:
    min_dist -- the minimum distance between image_path encoding and the encodings from the database
    identity -- string, the name prediction for the person on image_path
    """
    encoding = img_to_encoding(image, model)
    
    min_dist = 100
    identity = None
    
    # Loop over the database dictionary's names and encodings.
    for (name, db_encs) in database.items():
        
        # Compute L2 distance between the target "encoding" and the current "emb" from the database.
        dists = [np.linalg.norm(db_enc - encoding) for db_enc in db_encs]
        dist = np.min(dists)
        print('distance for %s is %s' %(name, dist))

        # If this distance is less than the min_dist, then set min_dist to dist, and identity to name
        if dist < min_dist:
            min_dist = dist
            identity = name
    
    if min_dist > 0.52:
        return None
    else:
        return str(identity)

def welcome_users(identities):
    """ Outputs a welcome audio message to the users """
    global ready_to_detect_identity

    if len(identities) == 1:
        welcome_message = 'I see one face. Hello, %s.' % identities
    else:
        welcome_message = 'I see ' + str(len(identities)) + ' faces. '
        for identity_id in range(len(identities)-1):
            welcome_message += 'I see %s, ' % identities[identity_id]
        welcome_message += 'and I see %s.' % identities[-1]
        say(welcome_message)
        print(welcome_message)

    # Allow the program to start detecting identities again
    time.sleep(5)
    ready_to_detect_identity = True

In [5]:
database = prepare_database()

error: OpenCV(4.0.0) /Users/travis/build/skvark/opencv-python/opencv/modules/imgproc/src/resize.cpp:3784: error: (-215:Assertion failed) !ssize.empty() in function 'resize'


In [9]:
database = {}

for file in glob.glob("images/*"):
    file = ''.join([i for i in file if not i.isdigit()])
    identity = os.path.splitext(os.path.basename(file))[0]


In [10]:
identity

'Subrot'

In [11]:
identity in database

False

In [15]:
file

'images/Subrot.png'

In [14]:
img_path_to_encoding(file, FRmodel)

error: OpenCV(4.0.0) /Users/travis/build/skvark/opencv-python/opencv/modules/imgproc/src/resize.cpp:3784: error: (-215:Assertion failed) !ssize.empty() in function 'resize'


In [33]:
database = {}

# load all the images of individuals to recognize into the database
for file_num in glob.glob("images/*"):
    print(file_num)
    identity = os.path.splitext(os.path.basename(file_num))[0]
    identity = ''.join([i for i in identity if not i.isdigit()])
    if identity in database:
        database[identity] = database[identity] + [img_path_to_encoding(file_num, FRmodel)]
    else:
        database[identity] = [img_path_to_encoding(file_num, FRmodel)]


images/Subrot24.png
images/Eric48.png
images/Eric60.png
images/Subrot18.png
images/Nico58.png
images/Nico59.png
images/Nico71.png
images/Eric75.png
images/Eric61.png
images/Subrot31.png
images/Eric49.png
images/Subrot25.png
images/Subrot8.png
images/Subrot27.png
images/Eric77.png
images/Eric63.png
images/Eric88.png
images/Nico73.png
images/Nico67.png
images/Nico98.png
images/Nico66.png
images/Nico72.png
images/Eric89.png
images/Eric62.png
images/Subrot26.png
images/Subrot32.png
images/Subrot9.png
images/Eric72.png
images/Eric66.png
images/Subrot36.png
images/Subrot22.png
images/Nico62.png
images/Nico89.png
images/Nico88.png
images/Nico63.png
images/Nico77.png
images/Eric98.png
images/Subrot23.png
images/Subrot37.png
images/Eric67.png
images/Eric73.png
images/Eric65.png
images/Eric71.png
images/Subrot21.png
images/Eric59.png
images/Subrot35.png
images/Nico49.png
images/Nico61.png
images/Nico75.png
images/Nico74.png
images/Nico60.png
images/Subrot34.png
images/Eric58.png
images/Eric70.pn

In [23]:
file_nums = glob.glob("images/*")

In [24]:
file_num = file_nums[0]

In [28]:
identity = os.path.splitext(os.path.basename(file_num))[0]
identity = ''.join([i for i in identity if not i.isdigit()])
database[identity] = [img_path_to_encoding(file_num, FRmodel)]

In [32]:
database['Subrot'][0][0]

array([-0.00502624,  0.11563899,  0.10302966,  0.08055655,  0.04786522,
        0.09058852, -0.01047758, -0.21145758,  0.07824475, -0.08698074,
       -0.00376674,  0.04527403, -0.0492423 ,  0.07970748, -0.0373813 ,
       -0.03539709,  0.03340322,  0.02649122, -0.09057584,  0.01418769,
       -0.00414894,  0.0757481 , -0.13213   ,  0.10599813, -0.00400234,
       -0.03750094, -0.1374872 ,  0.0836149 ,  0.05501855,  0.10377266,
        0.01811191,  0.16998237, -0.05788425,  0.00348586,  0.03582006,
        0.11841219, -0.00762442, -0.0467183 , -0.02028409, -0.02277073,
        0.04087079,  0.02053821, -0.04727909, -0.22694369,  0.11160266,
        0.14258184,  0.00825016,  0.01483681, -0.10642183, -0.11865205,
       -0.03519388,  0.13819292,  0.14502957, -0.05833407,  0.04039501,
        0.06369919, -0.09414969,  0.06669276, -0.10183578,  0.04110904,
       -0.01602299,  0.18485168,  0.0267094 , -0.08616544,  0.00476379,
        0.135763  ,  0.08840923,  0.02333755, -0.15997659,  0.11

In [5]:
import cv2

from faced.detector import FaceDetector
from faced.utils import annotate_image

face_detector = FaceDetector()

img = cv2.imread('test.png')
rgb_img = cv2.cvtColor(img.copy(), cv2.COLOR_BGR2RGB)

thresh = 0.9

# Receives RGB numpy image (HxWxC) and
# returns (x_center, y_center, width, height, prob) tuples. 
bboxes = face_detector.predict(rgb_img, thresh)

def crop(img,bbox):
    x, y, w, h, p = bbox
    x1 = int(x - w/2)
    y1 = int(y - h/2)
    x2 = int(x + w/2)
    y2 = int(y + h/2)
    height, width, channels = img.shape
    return img[max(0, y1):min(height, y2), max(0, x1):min(width, x2),:]

test_img = crop(rgb_img,bboxes[0])

ModuleNotFoundError: No module named 'faced'

absl-py==0.7.0
alabaster==0.7.12
anaconda-client==1.7.2
anaconda-navigator==1.9.6
anaconda-project==0.8.2
appdirs==1.4.3
appnope==0.1.0
appscript==1.0.1
asn1crypto==0.24.0
astor==0.7.1
astroid==2.1.0
atomicwrites==1.3.0
attrs==18.2.0
autograd==1.2
Automat==0.7.0
Babel==2.6.0
backcall==0.1.0
backports.os==0.1.1
backports.shutil-get-terminal-size==1.0.0
beautifulsoup4==4.7.1
bitarray==0.8.3
bleach==3.1.0
boto==2.49.0
certifi==2018.11.29
cffi==1.12.1
chardet==3.0.4
Click==7.0
cloudpickle==0.7.0
clyent==1.2.2
colorama==0.4.1
conda==4.6.7
conda-build==3.15.1
conda-verify==3.1.1
constantly==15.1.0
contextlib2==0.5.5
cryptography==2.5
cycler==0.10.0
Cython==0.29.5
cytoolz==0.9.0.1
dash==0.27.0
dash-core-components==0.30.2
dash-html-components==0.13.2
dash-renderer==0.14.1
dask==1.1.1
decorator==4.3.2
defusedxml==0.5.0
distributed==1.25.3
docutils==0.14
entrypoints==0.3
et-xmlfile==1.0.1
fastcache==1.0.2
filelock==3.0.10
Flask==1.0.2
Flask-Compress==1.4.0
Flask-Cors==3.0.7
future==0.17.1
gast=